## Semantic Textual Similarity

Задача семантическая близости -- это задача определения, насколько два текста похожи друг на друга по смыслу.

**План на сегодня:**
* Меры близости -- recap
* Нормы векторов -- recap
* Ранжирование
* DSSM

## Норма вектора

Норма -- это качественное измерение вектора. Обычно обозначается как $\| x \|$. Для нормы вектора соблюдаются следующие свойства:

- $\Vert \alpha x \Vert = |\alpha| \Vert x \Vert$
- $\Vert x + y \Vert \leq \Vert x \Vert + \Vert y \Vert$ (неравенство треугольника)
- Если $\Vert x \Vert = 0$ тогда $x = 0$

### Примеры
Самая популярная норма -- **евклидова норма** ($L_2$ норма):

$$\Vert x \Vert_2 = \sqrt{\sum_{i=1}^n |x_i|^2},$$

Евклидова норма является подклассом $p$-норм:

$$
 \Vert x \Vert_p = \Big(\sum_{i=1}^n |x_i|^p\Big)^{1/p}.
$$

Несколько интересных случаев:

- Норма Чебышева: 

$$
\Vert x \Vert_{\infty} = \max_i | x_i|
$$

- $L_1$ норма (или **манхэттенское расстояние**): 

$$
\Vert x \Vert_1 = \sum_i |x_i|
$$
  
- $L_0$ норма: $\Vert x \Vert_0 = $ количество ненулевых элементов.
  

In [16]:
import numpy as np

In [17]:
x = np.random.rand(100)

print('L0 norm =', np.linalg.norm(x, ord=0))
print('L1 norm =', np.linalg.norm(x, ord=1))
print('L2 norm =', np.linalg.norm(x, ord=2))
print('Норма Чебышева =', np.linalg.norm(x, ord=np.inf))

L0 norm = 100.0
L1 norm = 49.50485324688436
L2 norm = 5.729225789975424
Норма Чебышева = 0.9918537686056458


In [18]:
x = np.array([0, 0, 1, 1, 99])

print('L0 norm =', np.linalg.norm(x, ord=0))
print('L1 norm =', np.linalg.norm(x, ord=1))
print('L2 norm =', np.linalg.norm(x, ord=2))
print('Норма Чебышева =', np.linalg.norm(x, ord=np.inf))

L0 norm = 3.0
L1 norm = 101.0
L2 norm = 99.0101004948485
Норма Чебышева = 99.0


## Меры близости

Мера близости — это численная мера, показывающая степень схожести двух объектов. Как правило, она выражается в виде скалярной величины.

Пусть у нас есть два вектора $x$ и $y$. Как понять, насколько они похожи друг на друга?

* Посчитать угол $\theta$ между ними. $cos(\theta) = \frac{x^T y}{\| x \| \| y \|}$
* Посчитать норму их разницы $\| x - y \|$

In [19]:
x = np.random.normal(0, 0.001, 100)  # zero mean, and 0.001 standard deviation
y = np.random.normal(0, 0.001, 100)

print('Euclidean distance =', np.linalg.norm(x - y, ord=2))

Euclidean distance = 0.015414636388088382


In [20]:
x = np.random.normal(0, 5, 100)  # zero mean, and 5 standard deviation
y = np.random.normal(0, 5, 100)

print('Euclidean distance =', np.linalg.norm(x - y, ord=2))

Euclidean distance = 68.75526070902164


In [21]:
x = np.array([1, 1, 1])
y = np.array([1, 1, 1])

print('Euclidean distance =', np.linalg.norm(x - y, ord=2))

Euclidean distance = 0.0


<img src="https://cv4.litres.ru/pub/c/elektronnaya-kniga/cover_415/602545-anvar-bakirov-nlp-igry-v-kotoryh-pobezhdaut-zhenschiny.jpg" width="300">

## Ранжирование


Представьте, что у вас есть поисковая система, и на каждый запрос пользователя нужно выдавать правильные ссылки. Как это сделать?

Как это делает [Яндекс?](https://habr.com/ru/company/yandex/blog/314222/) Как это делали наши деды?

**Идея:** давайте введем меру близости и по ней будем ранжировать документы.

* Hamming расстояние
* Sørensen–Dice коэффициент
* Jaccard индекс
* Tversky индекс
* Меры близости на BoW, TF-IDF представлениях
* BM25

**Jaccard:**

$$
 J(A,B) = {{|A \cap B|}\over{|A \cup B|}} = {{|A \cap B|}\over{|A| + |B| - |A \cap B|}}.
$$


**BM25:**

$$
{\text{score}}(D,Q)=\sum _{{i=1}}^{{n}}{\text{IDF}}(q_{i})\cdot {\frac  {f(q_{i},D)\cdot (k_{1}+1)}{f(q_{i},D)+k_{1}\cdot (1-b+b\cdot {\frac  {|D|}{{\text{avgdl}}}})}}
$$

$$
{\text{IDF}}(q_{i})=\log {\frac  {N-n(q_{i})+0.5}{n(q_{i})+0.5}}
$$

Пусть от юзера приходит следующий запрос:
> рассказ в котором раздавили бабочку 

И мы по мере близости ранжируем наши документы (пример [отсюда](https://habr.com/ru/company/yandex/blog/314222/)):


|                  Заголовок страницы                 | BM25 | Нейронная модель |
|:---------------------------------------------------:|:----:|:----------------:|
| фильм в котором раздавили бабочку                   | 0.79 |             0.82 |
| и грянул гром википедия                             |    0 |             0.43 |
| брэдбери рэй википедия                              |    0 |             0.27 |
| машина времени роман википедия                      |    0 |             0.24 |
| домашнее малиновое варенье рецепт заготовки на зиму |    0 |             0.06 |


**Вывод:** модели/подходы прошлого века не устойчивы к перефразам.

In [22]:
from scipy.spatial.distance import jaccard
from sklearn.feature_extraction.text import CountVectorizer

In [23]:
# будем искать ближайшее предложение к первому предложению

sent1 = 'добрый вечер'
sent2 = 'добрый день'
sent3 = 'добрейший вечерок'

In [24]:
# bag-of-words

cv = CountVectorizer()
sentences = cv.fit_transform([sent1, sent2, sent3]).toarray()

sentences

array([[1, 0, 0, 0, 1],
       [0, 0, 1, 0, 1],
       [0, 1, 0, 1, 0]], dtype=int64)

In [25]:
# Jaccard-Needham dissimilarity

print('Jaccard 0-1 =', jaccard(sentences[0], sentences[1]))
print('Jaccard 0-2 =', jaccard(sentences[0], sentences[2]))

Jaccard 0-1 = 0.6666666666666666
Jaccard 0-2 = 1.0


In [26]:
# Использование буквенных n-грам почти всегда лучше
# bag-of-character n-grams

cv = CountVectorizer(ngram_range=(1, 3), analyzer='char')
sentences = cv.fit_transform([sent1, sent2, sent3]).toarray()

sentences.shape

(3, 59)

In [27]:
# Jaccard-Needham dissimilarity

print('Jaccard 0-1 =', jaccard(sentences[0], sentences[1]))
print('Jaccard 0-2 =', jaccard(sentences[0], sentences[2]))

Jaccard 0-1 = 0.6097560975609756
Jaccard 0-2 = 0.5510204081632653


"Добрейший вечерок" стал ближе, чем "добрый день"!

## DSSM and Beyond

А что NLP? Оказывается, подобным способом можно классифицировать тексты и делать многое другое, но сначала давайте познакомимся с идеями статьи [Learning Deep Structured Semantic Models for Web Search using Clickthrough Data](https://www.microsoft.com/en-us/research/publication/learning-deep-structured-semantic-models-for-web-search-using-clickthrough-data/) от Microsoft.

Ребята из Microsoft улучшали свой поисковик. В качестве данных у них были запросы пользователей и соответственные клики (не клики) на ссылки. 

### Model
Обозначим за $Q_i$ -- запрос пользователя и $D_j$ -- документ (веб-страницу).

Выходы моделей обозначим за $M_Q(Q_i)$ и $M_D(D_j)$. Наша цель -- обучить модель так, чтобы расстояние от запроса пользователя до правильного ответа (документа), было меньше, чем расстояние до неправильного.

$$
dist(M_Q(Q_i), M_D(D_i^+)) < dist(M_Q(Q_i), M_D(D_i^-))
$$

Мы собираем наборы из $(Q, D^+, D_1^-, D_2^-, \cdots, D_k^-)$ и проганяем их через следующую сеть:
<img src="https://raw.githubusercontent.com/v-liaha/v-liaha.github.io/master/assets/dssm.png" width="600">

В качестве функции потерь мы можем использовать:

$$
L(\Lambda) = - \log \prod_{Q, D^+} P(D^+|Q)
$$

## Практическая часть

Попробуем перенести наши идеи на TensorFlow.

<img src='https://scontent.fhrk1-1.fna.fbcdn.net/v/t1.0-9/37006042_518786041909234_5582352793441665024_n.png?_nc_cat=104&oh=8e36fa4907bb5f15c621c4c9b9215bc8&oe=5C8960E4' width="400">

Скачиваем данные [Quora Question Pairs](https://www.kaggle.com/quora/question-pairs-dataset)

**Описание данных:**

* id - the id of a training set question pair
* qid1, qid2 - unique ids of each question (only available in train.csv)
* question1, question2 - the full text of each question
* is_duplicate - the target variable, set to 1 if question1 and question2 have essentially the same meaning, and 0 otherwise.

In [31]:
import os
import re

import tensorflow as tf
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences

tf.reset_default_graph()

In [32]:
def tokenize_string(string):
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`’:]", " ", string)  
    string = re.sub(r"’", "'", string) 
    string = re.sub(r"`", "'", string) 
    string = re.sub(r"\'s", " \'s", string) 
    string = re.sub(r"\'ve", " \'ve", string) 
    string = re.sub(r"n\'t", " n\'t", string) 
    string = re.sub(r"\'re", " \'re", string) 
    string = re.sub(r"\'d", " \'d", string) 
    string = re.sub(r"\'ll", " \'ll", string) 
    string = re.sub(r",", " , ", string) 
    string = re.sub(r":", " : ", string) 
    string = re.sub(r"!", " ! ", string) 
    string = re.sub(r"\(", " ( ", string) 
    string = re.sub(r"\)", " ) ", string) 
    string = re.sub(r"\?", " ? ", string) 
    string = re.sub(r"\s{2,}", " ", string)    
    return string.strip().lower()

In [33]:
data = pd.read_csv('questions.csv', nrows=50000)
data = data[data['is_duplicate'] == 1]
data = data.dropna()
data = data.rename({'question1': 'query', 'question2': 'd+'}, axis=1)

In [34]:
data.head()

,id,qid1,qid2,query,d+,is_duplicate
5,5,11,12,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",1
7,7,15,16,How can I be a good geologist?,What should I do to be a great geologist?,1
11,11,23,24,How do I read and find my YouTube comments?,How can I see all my Youtube comments?,1
12,12,25,26,What can make Physics easy to learn?,How can you make physics easy to learn?,1
13,13,27,28,What was your first sexual experience like?,What was your first sexual experience?,1


In [35]:
num_negatives = 5

data['query'] = data['query'].apply(lambda x: tokenize_string(x))
data['d+'] = data['d+'].apply(lambda x: tokenize_string(x))

for i in range(num_negatives):
    data[f'd{i}-'] = np.random.permutation(data['d+'].values)

y = np.zeros((data.shape[0], num_negatives + 1), dtype=np.int64)
y[:, 0] = 1

Вопрос: почему так собирать негативные примеры плохо?

In [36]:
columns = ['query', 'd+'] + [f'd{i}-' for i in range(num_negatives)]

data[columns].sample(5)

,query,d+,d0-,d1-,d2-,d3-,d4-
10356,why does my 5 month old boxer puppy bark all n...,how do you stop a puppy from barking all night ?,how can india become cashless ?,what are some of the best contemporary books ?,what does a donald trump presidency mean for t...,how does google 's new nocaptcha recaptcha work ?,what will happen when the andromeda galaxy col...
48759,how is flirting healthy ?,is flirting good for health ?,who is the best escort service provider in ban...,can i use jio sim in 3g handset phone ?,what are the differences between chinese and w...,what were the major effects of the cambodia ea...,what are some shoes similar to chuck taylor sh...
42883,how big do you think the vr sector will be ?,how big will vr be ?,why did n't the ina get integrated in the indi...,what are your top three favorite books and why ?,can i substitute white sugar for brown ?,can meth be out of your system in 48 hours ?,what countries did yugoslavia split into ? whi...
38139,are dark matter and neutrinos at some level th...,are neutrinos dark matter ?,why ms dhoni retire from test ?,when and how did time begin ?,what are the best places to visit in india ?,jio bar code lost re generated ?,how it is like to be a pornstar ?
32194,why do tall guys love short girls ?,why do tall guys like short ladies ?,what are some things you should n't do or say ...,how can i overcome the fear of failure ?,how can i lose weight ?,what are the principles of buddhism ?,what is the secret of keeping a successful lon...


In [37]:
# фитим токенайзер

time_steps = 15
vocab_size = 5000

corpus = data['query'].tolist() + data['d+'].tolist()
tok = Tokenizer(num_words=vocab_size)
tok.fit_on_texts(corpus)

In [38]:
%%time

def vectorize(tokenizer, data, time_steps):
    data = tokenizer.texts_to_sequences(data)
    data = pad_sequences(data, maxlen=time_steps, padding='post')
    return data

X = {col: vectorize(tok, data[col].values, time_steps) for col in columns}

CPU times: user 2.18 s, sys: 4.9 ms, total: 2.19 s
Wall time: 2.18 s


In [39]:
X['query'].shape, X['d+'].shape

((18650, 15), (18650, 15))

In [40]:
# создаем отложенную выборку

ind_train, ind_eval = train_test_split(np.arange(len(data)), test_size=0.1, random_state=24)

X_train = {key: val[ind_train] for key, val in X.items()}
X_eval = {key: val[ind_eval] for key, val in X.items()}

y_train = y[ind_train]
y_eval = y[ind_eval]

In [41]:
# подача данных в модель

def input_fn(x, labels, params, is_training):
    dataset = tf.data.Dataset.from_tensor_slices((x, labels))

    if is_training:
        dataset = dataset.shuffle(buffer_size=params['buffer_size'])
        dataset = dataset.repeat(count=params['num_epochs'])

    dataset = dataset.batch(params['batch_size'])
    dataset = dataset.prefetch(buffer_size=2)
    return dataset

<img src='https://pp.userapi.com/c845018/v845018002/115e82/MAdbYTyJXnA.jpg' width="400">

In [42]:
def cosine_sim(x, y):
    """
    Calculate cosine similarity
    Args:
        x: [batch_size, emb_dim]
        y: [batch_size, emb_dim]
    Returns:
        cos_sim: tf.float64
    """
    xn = tf.nn.l2_normalize(x, axis=1)
    yn = tf.nn.l2_normalize(y, axis=1)
    cos_sim = tf.expand_dims(tf.reduce_sum(tf.multiply(xn, yn), axis=1), axis=-1)
    return cos_sim

In [43]:
# определяем архитектуру

def build_model(features, params, is_training):
    
    # Embedding matrix
    emb_matrix = tf.get_variable('embedding_matrix',
                                 shape=[params['vocab_size'], params['emb_size']],
                                 dtype=tf.float64)

    # Our architecture (try CNN/RNN/Whatever you want)
    def encode(x):
        embs = tf.nn.embedding_lookup(emb_matrix, x)
        max_pool = tf.reduce_max(embs, axis=1)
            
        with tf.name_scope('dense'):
            out = tf.layers.dense(max_pool, 64)
            
        return out
    
    # the same architecture for the query and documents
    encoded_features = {}        
    
    with tf.variable_scope('encoder'):
        encoded_features['query'] = encode(features['query'])
    
    for key, value in features.items():
        if key != 'query':
            with tf.variable_scope('encoder', reuse=True):
                encoded_features[key] = encode(value)
    
    # Calculating cosine similarities
    cos_sims = {}
    for key, value in encoded_features.items():
        if key != 'query':
            cos_sims[key] = cosine_sim(encoded_features['query'], encoded_features[key])
    
    cosine_similarities = [cos_sims['d+']]
    for k in range(len(features) - 2):
        cosine_similarities.append(cos_sims[f'd{k}-'])

    cosine_similarities = tf.concat(cosine_similarities, axis=1)
    
    return cosine_similarities

In [44]:
# определяем: оптимайзер, лосс, метрики, поведение модели в зависимости от mode

def model_fn(features, labels, mode, params):
    
    is_training = (mode == tf.estimator.ModeKeys.TRAIN)
    
    with tf.variable_scope('model'):
        logits = build_model(features, params, is_training)
        
    preds = tf.argmax(logits, axis=1)
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        predictions = {'preds': preds}
        return tf.estimator.EstimatorSpec(mode=mode,
                                          predictions=predictions)
    
    accuracy = tf.reduce_mean(tf.cast(tf.equal(preds, tf.constant(0, dtype=tf.int64)), tf.float64))
    loss = tf.losses.softmax_cross_entropy(onehot_labels=labels, logits=logits)
    
    if mode == tf.estimator.ModeKeys.EVAL:
        with tf.variable_scope('metrics'):
            eval_metrics = {'accuracy': tf.metrics.mean(accuracy)}
        
        return tf.estimator.EstimatorSpec(mode, loss=loss, eval_metric_ops=eval_metrics)
    
    tf.summary.scalar('accuracy', accuracy)
    tf.summary.scalar('loss', loss)
    
    optimizer = tf.train.AdamOptimizer()
    global_step = tf.train.get_global_step()
    train_op = optimizer.minimize(loss, global_step=global_step)
    
    return tf.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op)

In [ ]:
tf.reset_default_graph()

model_params = {
    'vocab_size': vocab_size,
    'emb_size': 150
}

config = tf.estimator.RunConfig(tf_random_seed=123,
                                model_dir='experiment_0',
                                save_summary_steps=5)

estimator = tf.estimator.Estimator(model_fn,
                                   params=model_params,
                                   config=config)

In [ ]:
params = {
    'batch_size': 128,
    'num_epochs': 5,
    'buffer_size': int(len(data) * 0.3)
}

estimator.train(lambda: input_fn(X_train, y_train, params=params, is_training=True))

In [ ]:
eval_results = estimator.evaluate(lambda: input_fn(X_eval, y_eval, params=params, is_training=False))

for key, value in eval_results.items():
    print(f'{key}: {value}')

**Вывод:** мы с большой точностью умеем среди $k$ вопросов находить тот, который является дубликатом к исходному.

## Triplet Loss

Для metric learning (similarity learning) больше подходит другой лосс, с которым мы сегодня познакомимся.

**Идея:** давайте придумаем такой лосс, который будет притягивать примеры из одного класса и отталкивать примеры из разных классов. Для этого из батча будет формировать триплеты: anchor, positive, negative (A, P, N), такие что

* A и P из одного класса
* A и N из разных классов

Тогда функция потерь выглядит следующим образом:

$$
L = \frac 1N \underset {Q, D^+, D^-} \sum max(0, \space m - sim[M_Q(Q), M_D(D^+)] + sim[M_Q(Q), M_D(D^-)] )
$$

**Интерпретация:** правильные документы должны быть ближе, чем неправильные, как минимум на величину *margin* -- $m$

Заметьте, модели для запросов пользователей и документов могут быть одинаковыми $M_Q = M_D$ (одинаковые веса)


До и после обучения:
<img src='https://cdn-images-1.medium.com/max/1600/0*_WNBFcRVEOz6QM7R.' width="400">

На примере задаче Question Answering:
<img src='https://raw.githubusercontent.com/yandexdataschool/nlp_course/master/resources/margin.png' width="400">

## Стратегии обучения

Стратегии формирования триплетов:
* Offline triplet mining (формируем триплеты каждую эпоху)
* Online triplet mining (формируем триплеты каждый батч)

Стратегии обучения для online mining:
* Batch all strategy (все примеры могут пойти в триплеты)
* Batch hard strategy (в триплеты идут только самые сложные примеры)

<img src='https://omoindrot.github.io/assets/triplet_loss/triplets.png' width="400">

Наивная реализация триплет лосса:

```python
anchor_output = ...    # shape [None, 128]
positive_output = ...  # shape [None, 128]
negative_output = ...  # shape [None, 128]

d_pos = tf.reduce_sum(tf.square(anchor_output - positive_output), 1)
d_neg = tf.reduce_sum(tf.square(anchor_output - negative_output), 1)

loss = tf.maximum(0.0, margin + d_pos - d_neg)
loss = tf.reduce_mean(loss)
```

## Inference

Хорошо, а как предсказывать? С помощью kNN!

<img src='https://pbs.twimg.com/media/DmVRIqrXcAAOvtH.jpg' width="400">

Плюсы kNN:
1. Существуют [множество](https://github.com/erikbern/ann-benchmarks) методов, которые позволяют искать очень быстро.
2. Качество на train set'е близко к 100%
3. Вы можете легко включать и выключать любые классы из предсказаний

### Как реализовать на TensorFlow?

В contrib ветке TensorFlow лежит реализованный [triplet loss](https://www.tensorflow.org/versions/r1.12/api_docs/python/tf/contrib/losses/metric_learning/triplet_semihard_loss). Попробуйте применить его на своей задаче!

Или посмотрите на [эту](https://github.com/omoindrot/tensorflow-triplet-loss) реализацию triplet loss'a с нуля

## Примеры задач

* Text classification
* Face identification
* Chitchat
* Image captioning
* Твоя идея?

<img src='https://omoindrot.github.io/assets/triplet_loss/triplet_loss.png' width="500">

## Papers/blog posts

* [FaceNet: A Unified Embedding for Face Recognition and Clustering](https://arxiv.org/abs/1503.03832)
* [Siamese Neural Networks for One-shot Image Recognition](https://www.cs.cmu.edu/~rsalakhu/papers/oneshot1.pdf)
* [In Defense of the Triplet Loss for Person Re-Identification](https://arxiv.org/abs/1703.07737)
* [OpenFace 0.2.0: Higher accuracy and halved execution time](http://bamos.github.io/2016/01/19/openface-0.2.0/)
* [Triplet Loss and Online Triplet Mining in TensorFlow](https://omoindrot.github.io/triplet-loss)

## Вопросы:

* В чем минусы триплет лосса? как исправить?
* В чем преимущество triplet loss'а на задаче классификации по сравнению с log-loss'ом, если классов миллиард?
* Сколько всего комбинаций триплетов в батче?